In [ ]:
# !pip install scanpy
import os
import numpy as np
import pandas as pd
import scanpy as sc
import time
from scipy.stats import pearsonr
#
#more 10x datasets 
#https://support.10xgenomics.com/single-cell-multiome-atac-gex/datasets/


#switch to keras for custom ae
#https://blog.keras.io/building-autoencoders-in-keras.html
#merging nerual networks
#https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [ ]:
print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')
#make a shortcut to emily's shared drive folder in your drive so you can access the data at
import os
os.listdir('/content/drive/My Drive/methyl_impute')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Peek_data.ipynb',
 'Project Proposal Brainstorming.gdoc',
 'VAE_impute (vanilla) (custom dropout).ipynb',
 'adult_cortex_CG-CH_OLDCOPY.h5ad',
 'adult_cortex_CG_CH.h5ad',
 'cg_counts.parq',
 'cg_covs.parq',
 'PredictionModel.ipynb',
 'tf_multiclass_prediction.ipynb',
 'classifier_results.gslides',
 'Presentation.gslides',
 'VAE_impute (vanilla).ipynb',
 'VAE_impute (variational).ipynb']

In [ ]:
adata=sc.read_h5ad('/content/drive/My Drive/methyl_impute/adult_cortex_CG_CH.h5ad')
adata

AnnData object with n_obs × n_vars = 11945 × 44772
    obs: 'sample', 'L1', 'L2', 'L3', 'true_batch', 'age', 'age_groups', 'leiden'
    var: 'batch'
    uns: 'L2_colors', 'L3_colors', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
#collect top marker genes

sc.tl.rank_genes_groups(adata, 'L3',n_genes=20)
# sc.pl.rank_genes_groups(adata, sharey=False)
markers=[]
for i in range(27):
    for j in range(20):
        if adata.uns['rank_genes_groups']['names'][j][i] not in markers:
            markers.append(adata.uns['rank_genes_groups']['names'][j][i])
"""adata2=adata[:,markers]
sc.pp.scale(adata2)
sc.pl.heatmap(adata2, markers, groupby='L3',vmax=1,vmin=-1)"""

KeyboardInterrupt: ignored

In [ ]:
adata=adata[:,markers]
adata

In [ ]:
#data matrix, X
X=np.array(adata.X)
print(X.shape)

In [ ]:
print(X.shape[1])

In [ ]:
class VAE:
  def __init__(self, input_shape,batch_size=256,optimizer=keras.optimizers.Adam(learning_rate=.001),epochs=50,
               recon_loss_function=tf.keras.losses.MeanSquaredError(),callback=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)):
    # self.mu=layers.Dense(32,name='mu')
    # self.sigma=layers.Dense(32,name='sigma')
    self.mu = None
    self.sigma = None

    self.input_shape=input_shape
    self.batch_size=batch_size
    self.n_epochs=epochs
    self.optimizer=optimizer
    self.recon_loss_function=recon_loss_function
    self.callback=callback
    # self.model=self.generate_model_architecture()
    # self.model.compile(optimizer=optimizer,loss=self.vae_loss)
    self.encoder=self.generate_encoder()
    self.encoder.compile(optimizer=optimizer,loss=self.vae_loss)
    self.sampler=self.perform_sampling()
    # self.sampler.compile(optimizer=optimizer,loss=self.vae_loss)
    self.decoder=self.generate_decoder()
    self.decoder.compile(optimizer=optimizer,loss=self.vae_loss)
    self.kappa=1

  def vae_loss(self, y_true, y_pred):
    recon = self.recon_loss_function(y_true, y_pred)
    kl = K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))
    return 200*recon + self.kappa*kl

  def sample_z(self, args):
    vae_mu, vae_sigma = args
    eps = K.random_normal(shape=(K.shape(vae_mu)[0], K.shape(vae_mu)[1]), mean=0., stddev=1., seed=42)
    return vae_mu + K.exp(vae_sigma/2)*eps

  def kl_loss(self):
    return K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))

  def generate_encoder(self):
    mC_input=keras.Input(shape=(self.input_shape,),name='mC_input')
    dense1=layers.Dense(128,activation='relu', name='dense1')(mC_input)
    dense2=layers.Dense(64,activation='relu', name='dense2')(dense1)
    vae_mu=layers.Dense(32,activation='linear')(dense2)
    vae_sigma=layers.Dense(32,activation='linear')(dense2)

    return Model(mC_input, (vae_mu, vae_sigma), name='encoder')

  def perform_sampling(self):
    vae_mu = keras.Input(shape=(32,))
    vae_sigma = keras.Input(shape=(32,))
    out=layers.Lambda(self.sample_z)([vae_mu, vae_sigma])
    return Model([vae_mu, vae_sigma], out, name='sampler')

  def generate_decoder(self):
    input_latent = keras.Input(shape=(32,))
    out_pred=layers.Dense(self.input_shape,activation='linear',name='out_pred')(input_latent)
    return Model(input_latent, out_pred, name='decoder')

  """def generate_model_architecture(self):
    mC_input=keras.Input(shape=(self.input_shape,),name='mC_input')
    dense1=layers.Dense(128,activation='relu', name='dense1')(mC_input)
    dense2=layers.Dense(64,activation='relu', name='dense2')(dense1)
    self.mu=self.mu(dense2)
    self.sigma=self.sigma(dense2)
    latent=layers.Lambda(self.sample_z, output_shape=(32,), name='latent')([self.mu, self.sigma])
    # latent=layers.Dense(32,activation='relu', name='latent')(dense2)
    out_pred=layers.Dense(self.input_shape,activation='linear',name='out_pred')(latent)

    return Model(mC_input,out_pred)"""


  def train_vae(self,train_dataset,test_dataset):
    train_acc_metric = keras.metrics.MeanSquaredError()
    test_acc_metric = keras.metrics.MeanSquaredError()

    epochs = self.n_epochs
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        print(self.kappa)
        start_time = time.time()

        # Iterate over the batches of the dataset.
        for step, (x_batch_train) in enumerate(train_dataset):
            with tf.GradientTape() as encoder_tape, tf.GradientTape() as decoder_tape:
                self.mu, self.sigma = self.encoder(x_batch_train, training=True)
                latent = self.sampler([self.mu, self.sigma])
                logits=self.decoder(latent, training=True)
                # logits = self.model(x_batch_train, training=True)
                loss_value = self.vae_loss(x_batch_train, logits)
                # print(loss_value)
            encoder_grads = encoder_tape.gradient(loss_value, self.encoder.trainable_weights)
            decoder_grads = decoder_tape.gradient(loss_value, self.decoder.trainable_weights)
            self.optimizer.apply_gradients(zip(encoder_grads, self.encoder.trainable_weights))
            self.optimizer.apply_gradients(zip(decoder_grads, self.decoder.trainable_weights))
            # grads = tape.gradient(loss_value, self.model.trainable_weights)
            # self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

            # Update training metric.
            train_acc_metric.update_state(x_batch_train, logits)

            # Log every 31 batches.
            if step % 31 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                #print("Seen so far: %d samples" % ((step + 1) * batch_size))

        # Display metrics at the end of each epoch.
        train_acc = train_acc_metric.result()
        print("Reconstruction Training acc over epoch: %.4f" % (float(train_acc),))
        print("KL Divergence: %.10f" % (float(self.kl_loss())))

        # Reset training metrics at the end of each epoch
        train_acc_metric.reset_states()

        # Run a validation loop at the end of each epoch.
        for x_batch_test in test_dataset:
            self.mu, self.sigma = self.encoder(x_batch_test, training=False)
            latent = self.sampler([self.mu, self.sigma])
            test_logits = self.decoder(latent, training=False)
            # test_logits = self.model(x_batch_test, training=False)
            # Update val metrics
            test_acc_metric.update_state(x_batch_test, test_logits)
        test_acc = test_acc_metric.result()
        test_acc_metric.reset_states()
        print("Validation acc: %.4f" % (float(test_acc),))
        #print("Time taken: %.2fs" % (time.time() - start_time))

        if self.kappa < 0.95:
            self.kappa += 0.25

  def predict(self,X):
    self.mu, self.sigma = self.encoder.predict(X)
    latent = self.sampler([self.mu, self.sigma])
    return self.decoder.predict(latent)

  def model_summary(self):
    print(self.model.summary())

  def visualize_model(self):
    #dunno how to make this show from the function lol
    keras.utils.plot_model(self.model, show_shapes=True, show_layer_names=True)

  def correlation_accuracy(self, real_data):
    reconstructed_data = self.predict(real_data)
    corrs = 0
    for i in range(len(reconstructed_data)):
        corrs += pearsonr(reconstructed_data[i], real_data[i])[0]

    print(corrs/len(reconstructed_data))

  def mse_error(self, real_data):
    reconstructed_data = self.predict(real_data)
    mse = 0
    for i in range(len(reconstructed_data)):
        mse += mean_squared_error(reconstructed_data[i], real_data[i])

    print(mse/len(reconstructed_data))




In [ ]:
# X_train, X_test = train_test_split(X, test_size=0.33, random_state=42,shuffle=True)
X_train, X_test = train_test_split(np.array(adata.X), test_size=0.33, random_state=42,shuffle=True)

In [ ]:
vae=VAE(X_train.shape[1])

In [ ]:
batch_size=256

train_dataset=tf.data.Dataset.from_tensor_slices(X_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_dataset = test_dataset.batch(batch_size)

In [ ]:
len(X_train)/batch_size

31.26171875

In [ ]:
vae.train_vae(train_dataset,test_dataset)


Start of epoch 0
1
Training loss (for one batch) at step 0: 37.9012
Training loss (for one batch) at step 31: 30.5923
Reconstruction Training acc over epoch: 0.1168
KL Divergence: 15.2886524200
Validation acc: 0.0778

Start of epoch 1
1
Training loss (for one batch) at step 0: 29.0215
Training loss (for one batch) at step 31: 12.9607
Reconstruction Training acc over epoch: 0.0565
KL Divergence: 7.2303476334
Validation acc: 0.0271

Start of epoch 2
1
Training loss (for one batch) at step 0: 12.4957
Training loss (for one batch) at step 31: 9.4576
Reconstruction Training acc over epoch: 0.0248
KL Divergence: 4.8570814133
Validation acc: 0.0238

Start of epoch 3
1
Training loss (for one batch) at step 0: 9.8642
Training loss (for one batch) at step 31: 8.9348
Reconstruction Training acc over epoch: 0.0225
KL Divergence: 4.1816916466
Validation acc: 0.0225

Start of epoch 4
1
Training loss (for one batch) at step 0: 8.4782
Training loss (for one batch) at step 31: 7.9475
Reconstruction Tr

In [ ]:
vae.predict(X_train)

In [ ]:
# vae.correlation_accuracy(X_train)
vae.correlation_accuracy(X_test)
# vae.mse_error(X_train)
vae.mse_error(X_test)

0.9663105203861073
0.008784314968768754
